# Documento con el modelo convolucional

En este documento se desarrollara y explicará el modelo convolucional creado en el proyecto. 

### Librerias y funciones 

In [ ]:
import keras
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.model_selection import train_test_split
import scipy.io as sio
from ipynb.fs.full.Funciones import trans
import pandas as pd
from keras.models import Sequential
from keras.layers import Conv1D, BatchNormalization, Dropout, GlobalAveragePooling1D, Dense
from sklearn.model_selection import GridSearchCV
import numpy as np
from sklearn.model_selection import GridSearchCV
import numpy as np
from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras.layers import Conv1D, BatchNormalization, Dropout, GlobalAveragePooling1D, Dense
from sklearn.model_selection import GridSearchCV

In [ ]:
def trans(archivo):


    import scipy.io as sio
    import pandas as pd
    from statistics import median
    from statistics import mean
    from math import isnan
    from itertools import filterfalse
    from matplotlib import pyplot as plt
    import numpy as np


    mat = sio.loadmat(archivo)
    matriz_4d = mat['Pxx_nodes']

    ka = []
    tau = []
    nodosc = []
    espectro = []

    for i in range(len(matriz_4d)):
        for j in range(len(matriz_4d[i])):
            for k in range(len(matriz_4d[i][j])):

                ka.append(i)
                tau.append(j)
                nodosc.append(k)
                valor = np.array(matriz_4d[i][j][k])
                espectro.append(valor)


    dicta = {'K': ka, 'Tau': tau, 'Nodos': nodosc, 'Espectro': espectro}

    df = pd.DataFrame(dicta)
    print(df.head())

    return df

### Cargar y tratar datos originales

In [ ]:
 
datos = trans('espectros.mat') #Nombre del archivo

datos['Cortada'] = datos['Espectro'].apply(lambda x: np.array(x[0:500])) #crear la señal con un alongitud de 500
datos1 = datos.copy()

datos['g4'] = np.select(
    [
        datos1['Tau'].isin(range(15)) & datos1['K'].isin(range(8)),
        datos1['Tau'].isin(range(15, 31)) & datos1['K'].isin(range(8)),
        datos1['Tau'].isin(range(15)) & datos1['K'].isin(range(8, 16)), 
        datos1['Tau'].isin(range(15, 31)) & datos1['K'].isin(range(8, 16))
    ],
    [0, 1, 2, 3]
) # Se crean las clases




### Separar los datos en Train y Test

In [ ]:
y_datos = np.array(list(datos['g4']))
x_datos = np.array(list(datos['Cortada']))

x_train, x_test, y_train, y_test = train_test_split(x_datos, y_datos, test_size=0.15, random_state=35)

x_train = x_train.reshape((x_train.shape[0], x_train.shape[1], 1))
x_test = x_test.reshape((x_test.shape[0], x_test.shape[1], 1))

num_classes = len(np.unique(y_train))

### Creación y ejecución del modelo

Para estimar los mejores hiperparámetros del modelo convolucinoal, se ha realizado un Grid Search con los siguientes valores: 
- 'filters': [50, 100, 200, 300]
- 'kernel_size': [2, 5, 7, 9]
- 'dropout_rate': [0, 0.1, 0.5, 0.75]

Se muestra las curvas de aprendizaje para cada modelo, sus resultados numéricos en el test y los resultados del mejor modelo hasta el moemnto

In [ ]:
def create_model(filters=300, kernel_size=7, dropout_rate=0.1):
    '''
    Función utilizada para crear el modelo. 
    Por defecto tiene filters=300, kernel_size=7, dropout_rate=0.1. 
    
    '''
    model = Sequential([
        Conv1D(filters=filters, kernel_size=kernel_size, padding="same", activation="relu"),
        BatchNormalization(),
        Dropout(dropout_rate),
        Conv1D(filters=filters, kernel_size=kernel_size, padding="same", activation="relu"),
        BatchNormalization(),
        Dropout(dropout_rate),
        GlobalAveragePooling1D(),
        Dense(num_classes, activation="softmax")
    ])
    model.compile(optimizer="adam", loss="sparse_categorical_crossentropy", metrics=["accuracy"])
    return model


param_grid = {'filters': [50, 100, 200, 300], 'kernel_size': [2, 5, 7, 9], 'dropout_rate': [0, 0.1, 0.5, 0.75]} #Grid Search


callbacks = [
    keras.callbacks.ModelCheckpoint(
        "best_model.keras", save_best_only=True, monitor="val_loss"
    ),
    keras.callbacks.ReduceLROnPlateau(
        monitor="val_loss", factor=0.5, patience=20, min_lr=0.0001
    ),
    keras.callbacks.EarlyStopping(monitor="val_loss", patience=50, verbose=1),
]




best_score = 0
best_params = None
for filters in param_grid['filters']:
    for kernel_size in param_grid['kernel_size']:
        for dropout_rate in param_grid['dropout_rate']:
            print(f"Training model with filters={filters}, kernel_size={kernel_size}, dropout_rate={dropout_rate}") #Indicación del modelo en el que se encuentra
            
            current_model = create_model(filters=filters, kernel_size=kernel_size, dropout_rate=dropout_rate) # Entrenamiento del modelo
            history = current_model.fit(x_train, y_train, epochs=350, batch_size=32,
                                    callbacks=callbacks,
                                    validation_split=0.45, verbose=0)
            
            model = keras.models.load_model("best_model.keras") #Cargamos el mejor modelo
            score = model.evaluate(x_test, y_test, verbose=0)[1]  # Calculamos su resultado en Test
            
            # Mostramos las curvas de aprendizaje para el Accuracy
            plt.plot(history.history['val_loss'], color='red')
            plt.plot(history.history['loss'],color='yellow')
            plt.title("val_loss")
            plt.show()

            # Mostramos las curvas de aprendizaje para el Loss
            plt.plot(history.history['val_accuracy'], color = 'red')
            plt.plot(history.history['accuracy'], color='yellow')
            plt.title("val_accuracy")
            plt.show()
            
            #Guardamos el mejor modelo y los parámetros que utiliza
            if score > best_score:
                best_score = score
                best_params = {'filters': filters, 'kernel_size': kernel_size, 'dropout_rate': dropout_rate}
                history_mejor = history
                print("Best score:", best_score)
                print("Best params:", best_params)
                

# Mostramso el mejor modelo y sus parámetros
print("Best scoreA:", best_score)
print("Best paramsA:", best_params)